<a href="https://colab.research.google.com/github/yukinaga/gnn/blob/main/section_3/03_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習
データセット「PubMed」を使い、モデルを訓練しましょう。  
モデルの構築、およびモデルの訓練のコードを記述してください。  

## Google ドライブとの連携  
（2022.12.10更新 バージョンアップによりPyTorch Geometricのインストール速度が向上したため、Google Driveを使う必要がなくなりました）   
~~以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。~~

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")

~~Googleドライブ上のパスを指定します。~~

In [ ]:
# dir_name = "Live/gnn_live"  # 好きなパスを設定してください
# package_path = "/content/drive/MyDrive/" + dir_name + "/packages/"

## PyTorch Geometricのインストール
GNN用のライブラリ「PyTorch Geometric」、および関連ライブラリをGoogle Driveのパスを指定してインストールします。  

In [ ]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

~~Google Driveに保存したパッケージをシステムに追加します。  ~~

In [ ]:
# import sys

# sys.path.append(package_path)  

## データセットの読み込み
生物医学文献のデータセット、「PubMed」を読み込みます。  

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="/tmp/PubMed", name="PubMed")
data = dataset[0]

グラフの情報を表示するための関数を設定します。

In [ ]:
def graph_info(data):

    print("ノードの数:", data.num_nodes)
    print("エッジの数:", data.num_edges)
    print("特徴量の数:", data.num_node_features)
    print("無向グラフか？:", data.is_undirected())
    print("孤立したノードが有るか？:", data.has_isolated_nodes())
    print("自己ループがあるか？:", data.has_self_loops())

    print()

    print("キー: ", data.keys)
    print("各ノードの特徴量")
    print(data["x"])
    print("各ノードのラベル")
    print(data["y"])
    print("各エッジ")
    print(data["edge_index"])

関数を使って、データセットの情報を表示します。  

In [ ]:
graph_info(data)

## モデルの構築
以下のセルにコードを追記し、2層からなる「Graph Convolutional Networks」を実装してださい。  
層の実装には、`GCNConv()`を利用します。  
https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # ------- 以下にコードを書く -------
 


        # ------- ここまで -------

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index

        # ------- 以下にコードを書く -------



        # ------- ここまで -------

        return x

net = GCN()
net.cuda()  #GPU対応

## 学習
「PubMed」の訓練データを使ってモデルを訓練しましょう。  
以下のセルにコードを追記してください。  

In [ ]:
from torch import optim

data = data.cuda()  # GPU対応

# 交差エントロピー誤差関数
loss_fnc = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters())

net.train()  # 訓練モード
for epoch in range(200):
    # ------- 以下にコードを書く -------






    # ------- ここまで -------

## モデルの評価

In [ ]:
net.eval()  # 評価モード
pred = net(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
accuracy = int(correct) / int(data.test_mask.sum())
print("正解率:", str(accuracy*100) + "%")

### 〇解答例
以下は解答例です。

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # ------- 以下にコードを書く -------
        self.conv1 = GCNConv(dataset.num_node_features, 32)
        self.relu = nn.ReLU()  # ReLU
        self.conv2 = GCNConv(32, dataset.num_classes)
        # ------- ここまで -------

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index

        # ------- 以下にコードを書く -------
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        # ------- ここまで -------

        return x

net = GCN()
net.cuda()  #GPU対応

In [ ]:
from torch import optim

data = data.cuda()  # GPU対応

# 交差エントロピー誤差関数
loss_fnc = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters())

net.train()  # 訓練モード
for epoch in range(200):
    # ------- 以下にコードを書く -------
    optimizer.zero_grad()
    out = net(data)
    loss = loss_fnc(out[data.train_mask], data.y[data.train_mask])

    loss.backward()
    optimizer.step()
    # ------- ここまで -------